#1. Import

In [1]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# pip install imblearn
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
cd /content/drive/MyDrive/Wi-Fi/WiFi_model

/content/drive/.shortcut-targets-by-id/1c-r1c3xc-gpB9RGgM00cUDA3mE8X-AN4/Wi-Fi/WiFi_model


In [ ]:
ls

CSI_cnn_yh.ipynb             kt_train.py                       __pycache__/
kt_cross_vali_input_data.py  LR0.0001_BATCHSIZE16_NHIDDEN200/


In [ ]:
from __future__ import print_function
import sklearn as sk
from sklearn.metrics import confusion_matrix
import matplotlib

#matplotlib.use("Agg")
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import sys
from sklearn.model_selection import KFold, cross_val_score
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# from utils import readCsiAmp_PCA
from kt_cross_vali_input_data import csv_import, DataSet


In [ ]:
keras = tf.keras

learning_rate = 0.0001  # 학습의 속도에 영향. 너무 크면 학습이 overshooting해서 학습에 실패하고, 너무 작으면 더디게 진행하다가 학습이 끝나 버린다.
training_iters = 2000
batch_size = 16
display_step = 100

# Network Parameters
window_size = 500
threshold = 60

n_input = 90  # WiFi activity data input (img shape: 90*window_size)
n_steps = window_size  # timesteps
n_hidden = 200  # hidden layer num of features original 200
n_classes = 6  # WiFi activity total classes

# Output folder
OUTPUT_FOLDER_PATTERN = "LR{0}_BATCHSIZE{1}_NHIDDEN{2}/"
output_folder = OUTPUT_FOLDER_PATTERN.format(learning_rate, batch_size, n_hidden)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)



#2. 모델링

##2-1. Way1_KNN

### 1. 모델링

In [ ]:
cd ..

/content/drive/.shortcut-targets-by-id/1c-r1c3xc-gpB9RGgM00cUDA3mE8X-AN4/Wi-Fi


In [ ]:
ls

CSI_cnn_yh.ipynb             kt_train.py                       __pycache__/
kt_cross_vali_input_data.py  LR0.0001_BATCHSIZE16_NHIDDEN200/


In [ ]:
# x_go_sleep, x_pass_out, x_just_lay, x_k1, x_k2
# blay, llay, rlay, rsit, move

# data import
x_go_sleep, x_pass_out, x_just_lay, x_k1, x_k2, y_go_sleep, y_pass_out, y_just_lay, y_k1, y_k2 = csv_import()
# d = readCsiAmp_PCA()

print(" go_sleep =", len(x_go_sleep), " just_lay=", len(x_just_lay),  " pass_out=", len(x_pass_out),
      "x_k1=", len(x_k1), " x_k2=", len(x_k2) )

csv file importing...
backlay finished... xx= (1500, 500, 90) yy= (1500, 6)
forlay finished... xx= (1500, 500, 90) yy= (1500, 6)
empty finished... xx= (1500, 500, 90) yy= (1500, 6)
onfloor finished... xx= (1500, 500, 90) yy= (1500, 6)
indanger finished... xx= (1500, 500, 90) yy= (1500, 6)
 go_sleep = 1500  just_lay= 1500  pass_out= 1500 x_k1= 1500  x_k2= 1500


In [ ]:
y_go_sleep

array([['blay'],
       ['blay'],
       ['blay'],
       ...,
       ['blay'],
       ['blay'],
       ['blay']], dtype=object)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=20)


In [ ]:
wifi_y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
kk = 2  # =========================================================================================
cnt = 0
# Launch the graph
for ii in range(kk * 10):
    cnt+= 1
    if cnt == 5:
      break
    i = ii % kk
    # Roll the data
    x_go_sleep_ = np.roll(x_go_sleep, int(len(x_go_sleep) / kk), axis=0)
    y_go_sleep_ = np.roll(y_go_sleep, int(len(y_go_sleep) / kk), axis=0)
    x_just_lay_ = np.roll(x_just_lay, int(len(x_just_lay) / kk), axis=0)
    y_just_lay_ = np.roll(y_just_lay, int(len(y_just_lay) / kk), axis=0)
    x_pass_out_ = np.roll(x_pass_out, int(len(x_pass_out) / kk), axis=0)
    y_pass_out_ = np.roll(y_pass_out, int(len(y_pass_out) / kk), axis=0)
    x_k1 = np.roll(x_k1, int(len(x_k1) / kk), axis = 0)
    y_k1 = np.roll(y_k1, int(len(y_k1) / kk), axis=0)
    x_k2 = np.roll(x_k2, int(len(x_k2) / kk), axis=0)
    y_k2 = np.roll(y_k2, int(len(y_k2) / kk), axis=0)


    wifi_x_train = np.r_[x_go_sleep[int(len(x_go_sleep) // kk):],
                         x_pass_out[int(len(x_pass_out) // kk):],
                         x_just_lay[int(len(x_just_lay) // kk):],
                         x_k1[int(len(x_k1) // kk):],x_k2[int(len(x_k2) // kk):]]

    wifi_y_train = np.r_[y_go_sleep[int(len(y_go_sleep) // kk):],
                         y_pass_out[int(len(y_pass_out) // kk):],
                         y_just_lay[int(len(y_just_lay) // kk):],
                         y_k1[int(len(y_k1) // kk):],y_k2[int(len(y_k2) // kk):]]

    # wifi_y_train = wifi_y_train[:, 1:] #분류 클래스를 한개 줄임?
    print(wifi_y_train.shape)


    wifi_x_validation = np.r_[x_go_sleep[int(len(x_go_sleep) // kk):],
                             x_pass_out[int(len(x_pass_out) // kk):],
                             x_just_lay[int(len(x_just_lay) // kk):],
                             x_k1[int(len(x_k1) // kk):],
                             x_k2[int(len(x_k2) // kk):]]

    wifi_y_validation = np.r_[y_go_sleep[int(len(y_go_sleep) // kk):],
                              y_pass_out[int(len(y_pass_out) // kk):],
                              y_just_lay[int(len(y_just_lay) // kk):],
                              y_k1[int(len(y_k1) // kk):],
                              y_k2[int(len(y_k2) // kk):]]

    # wifi_y_validation = wifi_y_validation[:, 1:]

    wifi_x_train = wifi_x_train.reshape(len(wifi_x_train), -1)
    #wifi_y_train = np.argmax(wifi_y_train, axis = 1).reshape(-1,1)

    wifi_x_validation = wifi_x_validation.reshape(len(wifi_x_validation), -1)
    #wifi_y_validation = np.argmax(wifi_y_validation, axis = 1).reshape(-1,1)
    # print(wifi_x_train.shape)
    print("shape:", wifi_x_train.shape)
    clf.fit(wifi_x_train, wifi_y_train)

pred = clf.predict(wifi_x_validation)


(3750, 1)
shape: (3750, 45000)


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(3750, 1)
shape: (3750, 45000)


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(3750, 1)
shape: (3750, 45000)


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(3750, 1)
shape: (3750, 45000)


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
pred

array(['blay', 'blay', 'blay', ..., 'move', 'move', 'move'], dtype=object)

###2. 성능검증

In [ ]:
wifi_y_validation

array([['blay'],
       ['blay'],
       ['blay'],
       ...,
       ['move'],
       ['move'],
       ['move']], dtype=object)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred, wifi_y_validation)

array([[727,   0,   0,   0, 156],
       [  7, 750,   0,  15,  73],
       [  9,   0, 750,   0,  84],
       [  7,   0,   0, 735,  20],
       [  0,   0,   0,   0, 417]])

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(wifi_y_validation, pred)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(pred, wifi_y_validation)

0.9010666666666667

##2-2. Way2_CNN

### 모델링

In [ ]:
leaky_relu = tf.nn.leaky_relu

def CNN(
        input_shape=None,
        classes = 6):
    model = tf.keras.Sequential([
        # 특징 추출 부분
        # Conv 1
        tf.keras.layers.Conv2D(filters=4,
                               kernel_size=(5, 5),
                               strides=3,
                               padding="valid",
                               activation=tf.keras.activations.relu,
                               input_shape=input_shape),
        # Max Pool 1
        tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                  strides=(2, 2),
                                  padding="valid"),
        tf.keras.layers.BatchNormalization(),

        # Conv 1
        tf.keras.layers.Conv2D(filters=16,
                               kernel_size=(3, 3),
                               strides=2,
                               padding="same",
                               activation=tf.keras.activations.relu,
                               input_shape=input_shape),
        # Max Pool 1
        tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                  strides=(2, 3),
                                  padding="valid"),
        tf.keras.layers.BatchNormalization(),

        # Conv 2
        tf.keras.layers.Conv2D(filters=8,
                               kernel_size=(3, 3),
                               strides=2,
                               padding="same",  # 크기 유지, 즉 여기서는 2임
                               activation=tf.keras.activations.relu),
        # Max Pool 2
        tf.keras.layers.MaxPool2D(pool_size=(3, 5),
                                  strides=(2, 3),
                                  padding="same"),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Flatten(),

        # 분류 층 부분
        # Fully connected layer 1
        tf.keras.layers.Dense(units=64,
                              activation=tf.keras.activations.relu),
        # tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),
        # Fully connected layer 2
        tf.keras.layers.Dense(units=32,
                              activation=tf.keras.activations.relu),
        # tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),

        tf.keras.layers.Dense(units=32,
                              activation=tf.keras.activations.relu, name="my_intermediate_layer"),
        # tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),

        # Fully connected layer 3
        tf.keras.layers.Dense(units=classes,
                              activation=tf.keras.activations.softmax)
    ])

    return model

In [ ]:
my_model = CNN((500, 90,1), n_classes)

print(my_model.summary())

my_model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 166, 29, 4)        104       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 82, 14, 4)        0         
 g2D)                                                            
                                                                 
 batch_normalization_12 (Bat  (None, 82, 14, 4)        16        
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 41, 7, 16)         592       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 20, 2, 16)        0         
 g2D)                                                            
                                                      

None


In [ ]:
my_model

In [ ]:
cd /content/drive/MyDrive/Wi-Fi/WiFi_model

[Errno 2] No such file or directory: '/content/drive/MyDrive/Wi-Fi/WiFi_model'
/


In [ ]:
ls

bin@      dev/   lib32@   mnt/                      python-apt/  srv/    usr/
boot/     etc/   lib64@   NGC-DL-CONTAINER-LICENSE  root/        sys/    var/
content/  home/  libx32@  opt/                      run/         tmp/
datalab/  lib@   media/   proc/                     sbin@        tools/


In [ ]:
# x_go_sleep, x_pass_out, x_just_lay, x_k1, x_k2
# blay, llay, rlay, rsit, move

# data import
x_go_sleep, x_pass_out, x_just_lay, x_k1, x_k2, y_go_sleep, y_pass_out, y_just_lay, y_k1, y_k2 = csv_import()
# d = readCsiAmp_PCA()

print("go_sleep =", len(x_go_sleep), " just_lay=", len(x_just_lay),  " pass_out=", len(x_pass_out),
      "x_k1=", len(x_k1), " x_k2=", len(x_k2) )

csv file importing...


FileNotFoundError: ignored

In [ ]:
# data shuffle

x_go_sleep, y_go_sleep = shuffle(x_go_sleep, y_go_sleep, random_state=0)
x_just_lay, y_just_lay = shuffle(x_just_lay, y_just_lay, random_state=0)
x_pass_out, y_pass_out = shuffle(x_pass_out, y_pass_out, random_state=0)
x_k1, y_k1 = shuffle(x_k1, y_k1, random_state=0)
x_k2, y_k2 = shuffle(x_k2, y_k2, random_state=0)

In [ ]:
testset_size = 0.25

x_go_sleep, x_go_sleep_test, y_go_sleep, y_go_sleep_test = train_test_split(
    x_go_sleep, y_go_sleep, test_size=testset_size, shuffle=False)

x_just_lay, x_just_lay_test, y_just_lay, y_just_lay_test = train_test_split(
    x_just_lay, y_just_lay, test_size=testset_size, shuffle=False)

x_pass_out, x_pass_out_test, y_pass_out, y_pass_out_test = train_test_split(
    x_pass_out, y_pass_out, test_size=testset_size, shuffle=False)

x_k1, x_k1_test, y_k1, y_k1_test = train_test_split(
    x_k1, y_k1, test_size=testset_size, shuffle=False)

x_k2, x_k2_test, y_k2, y_k2_test = train_test_split(
    x_k2, y_k2, test_size=testset_size, shuffle=False)

In [ ]:
test_x = np.r_[x_go_sleep_test, x_just_lay_test, x_pass_out_test, x_k1_test, x_k2_test]
test_y = np.r_[y_go_sleep_test, y_just_lay_test, y_pass_out_test, y_k1_test, y_k2_test]
test_y = test_y[:, 1:]

In [ ]:
print(wifi_x_train.shape, wifi_y_train.shape)

(2815, 45000) (2815, 1)


In [ ]:
kk = 2  # =========================================================================================
cnt = 0
# Launch the graph
for ii in range(kk * 10):
    cnt+= 1
    if cnt == 5:
      break
    i = ii % kk
    # Roll the data
    x_go_sleep_ = np.roll(x_go_sleep, int(len(x_go_sleep) / kk), axis=0)
    y_go_sleep_ = np.roll(y_go_sleep, int(len(y_go_sleep) / kk), axis=0)
    x_just_lay_ = np.roll(x_just_lay, int(len(x_just_lay) / kk), axis=0)
    y_just_lay_ = np.roll(y_just_lay, int(len(y_just_lay) / kk), axis=0)
    x_pass_out_ = np.roll(x_pass_out, int(len(x_pass_out) / kk), axis=0)
    y_pass_out_ = np.roll(y_pass_out, int(len(y_pass_out) / kk), axis=0)
    x_k1 = np.roll(x_k1, int(len(x_k1) / kk), axis = 0)
    y_k1 = np.roll(y_k1, int(len(y_k1) / kk), axis=0)
    x_k2 = np.roll(x_k2, int(len(x_k2) / kk), axis=0)
    y_k2 = np.roll(y_k2, int(len(y_k2) / kk), axis=0)


    wifi_x_train = np.r_[x_go_sleep[int(len(x_go_sleep) // kk):],
                         x_pass_out[int(len(x_pass_out) // kk):],
                         x_just_lay[int(len(x_just_lay) // kk):],
                         x_k1[int(len(x_k1) // kk):],x_k2[int(len(x_k2) // kk):]]

    wifi_y_train = np.r_[y_go_sleep[int(len(y_go_sleep) // kk):],
                         y_pass_out[int(len(y_pass_out) // kk):],
                         y_just_lay[int(len(y_just_lay) // kk):],
                         y_k1[int(len(y_k1) // kk):],y_k2[int(len(y_k2) // kk):]]

    # wifi_y_train = wifi_y_train[:, 1:] #분류 클래스를 한개 줄임?

    wifi_x_validation = np.r_[x_go_sleep[:int(len(x_go_sleep) // kk)],
                             x_pass_out[:int(len(x_pass_out) // kk)],
                             x_just_lay[:int(len(x_just_lay) // kk)],
                             x_k1[:int(len(x_k1) // kk)],
                             x_k2[:int(len(x_k2) // kk)]]

    wifi_y_validation = np.r_[y_go_sleep[:int(len(y_go_sleep) // kk)],
                              y_pass_out[:int(len(y_pass_out) // kk)],
                              y_just_lay[:int(len(y_just_lay) // kk)],
                              y_k1[:int(len(y_k1) // kk)],
                              y_k2[:int(len(y_k2) // kk)]]

    # wifi_y_validation = wifi_y_validation[:, 1:]

    wifi_x_train = tf.expand_dims(wifi_x_train, axis = -1)

    wifi_x_validation = tf.expand_dims(wifi_x_validation, axis = -1)

    # wifi_x_train = wifi_x_train.reshape(len(wifi_x_train), -1)

    # wifi_y_train = np.argmax(wifi_y_train, axis = 1).reshape(-1,1)

    # wifi_x_validation = wifi_x_validation.reshape(len(wifi_x_validation), -1)
    # wifi_y_validation = np.argmax(wifi_y_validation, axis = 1).reshape(-1,1)
    # print(wifi_x_train.shape)

    print("shape:", wifi_x_train.shape)
    print("shape:", wifi_y_train.shape)
    my_model.fit(wifi_x_train, wifi_y_train)

history = my_model.fit(wifi_x_train, wifi_y_train,
                        epochs=6,  # =========================================================================
                        validation_data=(wifi_x_validation, wifi_y_validation),
                        batch_size=batch_size)

my_model.save('cnn_wifi.h5')

shape: (2815, 500, 90, 1)
shape: (2815, 6)
88/88 [==============================] - 3s 36ms/step - loss: 0.0766 - accuracy: 0.9798
shape: (2815, 500, 90, 1)
shape: (2815, 6)
88/88 [==============================] - 3s 33ms/step - loss: 0.0570 - accuracy: 0.9847
shape: (2815, 500, 90, 1)
shape: (2815, 6)
88/88 [==============================] - 3s 29ms/step - loss: 0.0654 - accuracy: 0.9826
shape: (2815, 500, 90, 1)
shape: (2815, 6)
88/88 [==============================] - 3s 29ms/step - loss: 0.0774 - accuracy: 0.9755
Epoch 1/6
176/176 [==============================] - 4s 26ms/step - loss: 0.1102 - accuracy: 0.9652 - val_loss: 0.0433 - val_accuracy: 0.9872
Epoch 2/6
176/176 [==============================] - 4s 23ms/step - loss: 0.1074 - accuracy: 0.9631 - val_loss: 1.7055 - val_accuracy: 0.6103
Epoch 3/6
176/176 [==============================] - 4s 23ms/step - loss: 0.1227 - accuracy: 0.9616 - val_loss: 0.3861 - val_accuracy: 0.8701
Epoch 4/6
176/176 [==============================]

###성능검증

In [ ]:
pred

In [ ]:
y_pred = np.argmax(pred, axis = 1).reshape(-1,1)

In [ ]:
wifi_y_validation = np.argmax(wifi_y_validation, axis = 1).reshape(-1,1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, wifi_y_validation)

## 2-3. SVM

###모델링

In [ ]:
from sklearn.svm import SVC

model_svc = SVC(kernel='rbf', C=6, gamma=0.1)

In [ ]:
kk = 2  # =========================================================================================
cnt = 0
# Launch the graph
for ii in range(kk * 10):
    cnt+= 1
    if cnt == 5:
      break
    i = ii % kk
    # Roll the data
    x_go_sleep_ = np.roll(x_go_sleep, int(len(x_go_sleep) / kk), axis=0)
    y_go_sleep_ = np.roll(y_go_sleep, int(len(y_go_sleep) / kk), axis=0)
    x_just_lay_ = np.roll(x_just_lay, int(len(x_just_lay) / kk), axis=0)
    y_just_lay_ = np.roll(y_just_lay, int(len(y_just_lay) / kk), axis=0)
    x_pass_out_ = np.roll(x_pass_out, int(len(x_pass_out) / kk), axis=0)
    y_pass_out_ = np.roll(y_pass_out, int(len(y_pass_out) / kk), axis=0)
    x_k1 = np.roll(x_k1, int(len(x_k1) / kk), axis = 0)
    y_k1 = np.roll(y_k1, int(len(y_k1) / kk), axis=0)
    x_k2 = np.roll(x_k2, int(len(x_k2) / kk), axis=0)
    y_k2 = np.roll(y_k2, int(len(y_k2) / kk), axis=0)


    wifi_x_train = np.r_[x_go_sleep[int(len(x_go_sleep) // kk):],
                         x_pass_out[int(len(x_pass_out) // kk):],
                         x_just_lay[int(len(x_just_lay) // kk):],
                         x_k1[int(len(x_k1) // kk):],x_k2[int(len(x_k2) // kk):]]

    wifi_y_train = np.r_[y_go_sleep[int(len(y_go_sleep) // kk):],
                         y_pass_out[int(len(y_pass_out) // kk):],
                         y_just_lay[int(len(y_just_lay) // kk):],
                         y_k1[int(len(y_k1) // kk):],y_k2[int(len(y_k2) // kk):]]

    # wifi_y_train = wifi_y_train[:, 1:] #분류 클래스를 한개 줄임?
    print(wifi_y_train.shape)


    wifi_x_validation = np.r_[x_go_sleep[int(len(x_go_sleep) // kk):],
                             x_pass_out[int(len(x_pass_out) // kk):],
                             x_just_lay[int(len(x_just_lay) // kk):],
                             x_k1[int(len(x_k1) // kk):],
                             x_k2[int(len(x_k2) // kk):]]

    wifi_y_validation = np.r_[y_go_sleep[int(len(y_go_sleep) // kk):],
                              y_pass_out[int(len(y_pass_out) // kk):],
                              y_just_lay[int(len(y_just_lay) // kk):],
                              y_k1[int(len(y_k1) // kk):],
                              y_k2[int(len(y_k2) // kk):]]

    # wifi_y_validation = wifi_y_validation[:, 1:]

    wifi_x_train = wifi_x_train.reshape(len(wifi_x_train), -1)
    #wifi_y_train = np.argmax(wifi_y_train, axis = 1).reshape(-1,1)

    wifi_x_validation = wifi_x_validation.reshape(len(wifi_x_validation), -1)
    #wifi_y_validation = np.argmax(wifi_y_validation, axis = 1).reshape(-1,1)
    # print(wifi_x_train.shape)
    print("shape:", wifi_x_train.shape)
    model_svc.fit(wifi_x_train, wifi_y_train)

pred = model_svc.predict(wifi_x_validation)


In [ ]:
pred

In [ ]:
wifi_y_validation = np.argmax(wifi_y_validation, axis = 1).reshape(-1,1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred, wifi_y_validation)